# Plot GDC position in Earth Centered Inertial and Magnetic (Apex) Coordinates

In [ ]:
import numpy as np
import pandas as pd
#Load data written out in Convert notebook using Pandas
df = pd.read_csv('GDC_Ephemeris_G1_Phase1a_Apex.csv',parse_dates=[0],index_col=0)
df.head(3)

In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from polar_plot import latlt2polar,polar2dial

def draw_3d_earth(ax):
    """Draw a transparent sphere on an mplot3d axes object"""
    re=6371.2
    phi = np.linspace(0, 2 * np.pi, 100)
    theta = np.linspace(0, np.pi, 100)
    xm = re * np.outer(np.cos(phi), np.sin(theta))
    ym = re * np.outer(np.sin(phi), np.sin(theta))
    zm = re * np.outer(np.ones(np.size(phi)), np.cos(theta))
    ax.plot_surface(xm, ym, zm,alpha=.5)
 
istep=int(91*60/30) #91 minutes (very rough orbital period) at 30 second cadence
ist=0
ied=istep
iplot=1
maxplots=10
while ied < df.index.size and iplot<maxplots:
    
    subdf = df.iloc[ist:ied]
    
    f3d = plt.figure(figsize=(10,8),dpi=300)
    fNS = plt.figure(figsize=(8,4),dpi=300)
    ax3d = f3d.add_subplot(111,projection='3d')
    axN = fNS.add_subplot(121,projection='polar')
    axS = fNS.add_subplot(122,projection='polar')
    polar2dial(axN)
    polar2dial(axS)
    axN.set(title='North')
    axS.set(title='South')

    cmap='rainbow'

    ind = np.arange(subdf.index.size)
    
    draw_3d_earth(ax3d)
    mappable = ax3d.scatter(subdf['x'],subdf['y'],subdf['z'],s=3,c=ind,cmap=cmap,zorder=10)
    f3d.colorbar(mappable)

    for mask,ax,hemi in [(subdf['apex110lat']>50,axN,'N'),(subdf['apex110lat']<-50,axS,'S')]:
        r,th = latlt2polar(subdf['apex110lat'][mask],subdf['mlt'][mask],hemi)
        ax.scatter(th,r,3,ind[mask],cmap=cmap,vmin=ind[0],vmax=ind[-1])
    
    for f in [f3d,fNS]:
        f.suptitle('{}-{}'.format(pd.to_datetime(subdf.index.values[0]),pd.to_datetime(subdf.index.values[-1])))
    
    plt.show()
    
    ist+=istep
    ied+=istep
    iplot+=1

NameError: name 'df' is not defined